In [13]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [171]:
xy = np.genfromtxt('data-titanic_modified.csv',
                   delimiter=',', dtype=np.float32,
                   skip_header=1, filling_values=0
                   )

In [188]:
age_fare = xy[:, 1:3]
others = xy[:, 3:-1]
age_fare_n = age_fare/ age_fare.max(axis=0)
x_data = np.concatenate((age_fare_n, others), axis=1)
x_data[0].shape

(26,)

In [189]:
class TitanicDataset(Dataset):
    def __init__(self):
#         xy = np.loadtxt('data-titanic_modified.csv',
#                         delimiter=',', dtype=np.float32, 
#                         skiprows=1)
        xy = np.genfromtxt('data-titanic_modified.csv',
                           delimiter=',', dtype=np.float32,
                           skip_header=1, filling_values=0)
        self.len = xy.shape[0]
        ## normalize age & fare
        age_fare = xy[:, 1:3]
        others = xy[:, 3:-1]
        age_fare_n = age_fare/ age_fare.max(axis=0)
        x_data_raw = np.concatenate((age_fare_n, others), axis=1)

        self.x_data = torch.from_numpy(x_data_raw)
#         self.x_data = torch.from_numpy(xy[:, 1:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

dataset = TitanicDataset()
train_loader = DataLoader(dataset=dataset,
                         batch_size=64,
                         shuffle=True,
                         num_workers=2)

In [190]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(26, 16)
        self.l2 = torch.nn.Linear(16, 8)
        self.l3 = torch.nn.Linear(8, 1)
        
        self.activation = torch.nn.Sigmoid()
        
    def forward(self, x):
        out1 = self.activation(self.l1(x))
        out2 = self.activation(self.l2(out1))
        y_pred = self.activation(self.l3(out2))
        return y_pred


model = Model()
criterion = torch.nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer = torch.optim.Rprop(model.parameters(), lr=0.1)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=0.1)

In [191]:
for epoch in range(200):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
#         print(epoch, i, loss.data[0])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
    print(epoch, running_loss)

0 12.266276746988297
1 10.368754804134369
2 10.11886990070343
3 10.139748692512512
4 10.134407371282578
5 10.027240544557571
6 10.207036852836609
7 10.228344917297363
8 10.128618121147156
9 10.115591704845428
10 10.129667580127716
11 10.172004997730255
12 10.282431840896606
13 10.07414710521698
14 10.136689096689224
15 10.065505981445312
16 10.064886897802353
17 10.290424704551697
18 10.150776088237762
19 10.148715049028397
20 10.055893450975418
21 10.10054162144661
22 10.245718240737915
23 9.986223444342613
24 10.126644372940063
25 10.067294239997864
26 10.106126189231873
27 10.068854123353958
28 10.052556663751602
29 10.193464785814285
30 10.058157086372375
31 10.06968891620636
32 10.095128983259201
33 10.178996443748474
34 10.11747482419014
35 10.031634658575058
36 10.207082569599152
37 10.069390386343002
38 10.12829715013504
39 10.144826173782349
40 10.11453241109848
41 10.085858255624771
42 10.12531852722168
43 10.100275367498398
44 10.194907486438751
45 10.127004623413086
46 10.1